# Visualizar Embeddings
Utiliza este notebook para generar embeddings de dimensionalidad reducida (2D o 3D) para poder graficarlos.

In [1]:
import os
from os import listdir
from os.path import isfile, join
import pathlib
import torch
from sklearn.manifold import TSNE
import numpy as np

# Seteo el path al root del proyecto
dev_folder = 'dev'
folders = os.getcwd().split('/')
if (len(folders) == 1):
    folders = folders[0].split('\\')
if(folders[-1] == dev_folder):
    os.chdir('../')
print(os.getcwd())

/home/mrbeast/TPI-RNP-Palabras-Relacionadas-ISI


In [2]:
from torch import nn

class SkipGram(nn.Module):
    def __init__(self, vocabulary, embed_size):
        super().__init__()
        self.central_embedding = nn.Embedding(num_embeddings=len(vocabulary)+1,
                                embedding_dim=embed_size, padding_idx=len(vocabulary))
        self.context_embedding = nn.Embedding(num_embeddings=len(vocabulary)+1,
                                embedding_dim=embed_size, padding_idx=len(vocabulary))

    def forward(self, center, contexts_and_negatives):
        v = self.central_embedding(center)
        u = self.context_embedding(contexts_and_negatives)
        pred = torch.bmm(v, u.permute(0, 2, 1))
        return pred

In [3]:
import ast

def get_vocabulary(vocabularies_path, vocabulary):
    with open(vocabularies_path + "/" + vocabulary + ".txt", "rb") as cf:
        lines = cf.read().decode("utf-8").split("\n")
        return [ast.literal_eval(l) for l in lines]

In [11]:
def get_embeddings_reduct(vocabulary, skip_gram, dim_key, embeddings_path):
    if not dim_key in ["2d", "3d"]:
        print("dim_key Format Error: dim_key must be either 2d or 3d")
    dims = { "2d": 2, "3d": 3 }

    embeddings_weight = skip_gram.central_embedding.weight.data
    print(embeddings_weight.size())
    tsne = TSNE(dims[dim_key])

    vocab_emb_red = tsne.fit_transform(embeddings_weight[0:len(vocabulary)])
    data = np.insert(vocab_emb_red, [0], [[i] for i in range(len(vocab_emb_red))], axis=1)
    print(data[100])
    np.savetxt(f"{embeddings_path}/embeddings-{dim_key}.csv", data, delimiter=',')


In [12]:
# Filenames for vocab and model
selected_vocabulary = "vocab_0"
selected_model = "dataset-100-256-256.pt"

# Folders for vocab, model and embeddings
vocabularies_path = "./data/vocabularies"
models_path = "./data/models"
out_embeddings_path = "./data/embeddings"

vocabulary = get_vocabulary(vocabularies_path, selected_vocabulary)
print(len(vocabulary))
isinet = SkipGram(vocabulary, 256)
isinet.load_state_dict(torch.load(models_path + "/" + selected_model, weights_only=True))

get_embeddings_reduct(vocabulary, isinet, "2d", out_embeddings_path)
get_embeddings_reduct(vocabulary, isinet, "3d", out_embeddings_path)

441
torch.Size([442, 256])
[100.        -15.111571    5.8227644]
torch.Size([442, 256])
[100.         6.772122  22.009706  62.66788 ]


In [6]:
labels = [("hola", "mundo"), ("buenas", "tardes"), ("buenas")]
search_for = "buenas"
terms = tuple([word.strip() for word in search_for.split(',')])
print(terms, labels[1])

try:
    concept = tuple(terms)
    search_idx = labels.index(concept)
    print(search_idx)
except ValueError:
    print("not here buddy")


('buenas',) ('buenas', 'tardes')
not here buddy


In [7]:
import ast
def idx_to_label(idx):
    file = "./data/vocabularies/vocab_0.txt"
    with open(file, "rb") as cf:
        lines = cf.read().decode("utf-8").split("\n")
        vocabulary = [ast.literal_eval(l) for l in lines]
    return vocabulary[idx]

idx_to_label(0)

('arquitectura',)

In [9]:
file = f"./data/embeddings/embeddings-3d.csv"
data = np.loadtxt(file, delimiter=",").tolist()
last_idx = int(data[-1][0])
print(len(vocabulary), "FUCKING PRINT",len(data))
# first index of each row contains the label
labels = [idx_to_label(int(d[0])) for d in data]
data = np.array([d[1:] for d in data])
print(last_idx)
print(labels[-1], idx_to_label(last_idx))

441 FUCKING PRINT 442


IndexError: list index out of range